In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
import keras
import tensorflow as tf
import cv2
from keras.models import Model, Sequential
from keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D, InputLayer
import PIL
import os
from keras.optimizers import Adam
from keras import applications
from keras.layers import Input
from keras.applications.resnet50 import ResNet50
from keras.layers import GlobalAveragePooling2D
from sklearn.metrics import confusion_matrix,classification_report
from shutil import copyfile, copy2
from keras.callbacks import LearningRateScheduler

In [ ]:
from zipfile import ZipFile
# After uploading a file named "Set1&2.zip" to Google Colaboratory, load it. Set1&2 has roughly 40,000 images. 35,000 in train, 5,000 in test. 
file_name = "Set1&2.zip"

with ZipFile(file_name, 'r') as zip:
	zip.extractall()
	print("Done")
  
#with augmentation
from keras.preprocessing.image import ImageDataGenerator
generator = ImageDataGenerator(rescale=1./255,
                               zoom_range=0.2, # Zoom in by 20% max
                               horizontal_flip=True, # Allo horizontal flipping
                               fill_mode='nearest')
image_shape = (50,50,3)

generator_noaug = ImageDataGenerator(rescale=1./255)
# Training Dataset (With Shuffle)
pathtrain = "/content/Set1&2/Train"
firsttrain = generator.flow_from_directory(
        pathtrain,
        target_size=image_shape[:2],
        batch_size=128,
        shuffle=True)
# Test Dataset (Without Shuffl)
pathtest = "/content/Set1&2/Test"
firsttest = generator_noaug.flow_from_directory(
        pathtest,
        target_size=image_shape[:2],
        batch_size=64,
        shuffle=False)

# Assign Class Weights
from sklearn.utils.class_weight import compute_class_weight
class_weight = compute_class_weight(class_weight='balanced',
                                    classes=np.unique(firsttrain.classes),
                                    y=firsttrain.classes)
class_weight

In [ ]:
# The same training dataset but without shuffle (so wrong images can be identified)
image_shape = (50,50,3)
validate_first_train = generator.flow_from_directory(
        pathtrain,
        target_size=image_shape[:2],
        batch_size=64,
        shuffle=False)

In [ ]:
# Create directories to store images for MetaNet training
# define the names of the directories to be created
path1 = "/content/Resnet2"
path2 = "/content/Resnet2/Benign"
path3 = "/content/Resnet2/Malignant"

path4 = "/content/Resnet3"
path5 = "/content/Resnet3/Benign"
path6 = "/content/Resnet3/Malignant"

path7 = "/content/Resnet4"
path8 = "/content/Resnet4/Benign"
path9 = "/content/Resnet4/Malignant"

# ONLY RUN THIS CODE ONCE BC It makes the directories
import os
import shutil
shutil.rmtree(path2,path3)
try:
  os.mkdir(path1)
  os.mkdir(path2)
  os.mkdir(path3)
  os.mkdir(path4)
  os.mkdir(path5)
  os.mkdir(path6)  
  os.mkdir(path7)
  os.mkdir(path8)
  os.mkdir(path9)

In [ ]:
# Create new directories for the Ensemble Training
path1b = "/content/Resnet2b"
path2b = "/content/Resnet2b/Benign"
path3b = "/content/Resnet2b/Malignant"

path4b = "/content/Resnet3b"
path5b = "/content/Resnet3b/Benign"
path6b = "/content/Resnet3b/Malignant"

path7b = "/content/Resnet4b"
path8b = "/content/Resnet4b/Benign"
path9b = "/content/Resnet4b/Malignant"

import os
import shutil
#shutil.rmtree(path2,path3)
try:
  os.mkdir(path1b)
  os.mkdir(path2b)
  os.mkdir(path3b)
  os.mkdir(path4b)
  os.mkdir(path5b)
  os.mkdir(path6b)  
  os.mkdir(path7b)
  os.mkdir(path8b)
  os.mkdir(path9b)

In [ ]:
# Load all MetaNet Constituent Models (after training)
from keras.models import load_model
resnet1 = load_model('resnet1.h5')
resnet2 = load_model('resnet2_finetune.h5')
resnet3 = load_model('resnet3_finetune.h5')

In [ ]:
# Find ROC Value of MetaNet
fpr_meta, tpr_meta, thresholds_meta = metrics.roc_curve(y_test_vals2,final_predictions_ann)
print(metrics.auc(fpr_meta, tpr_meta))

In [ ]:
model1_pred_vals = resnet1_refinetune.predict_generator(validate_first_train,validate_first_train.samples/validate_first_train.batch_size)
model2_pred_vals = resnet2.predict_generator(validate_first_train,validate_first_train.samples/validate_first_train.batch_size)
model3_pred_vals = resnet3.predict_generator(validate_first_train,validate_first_train.samples/validate_first_train.batch_size)
model4_pred_vals = resnet1.predict_generator(validate_first_train,validate_first_train.samples/validate_first_train.batch_size)

In [ ]:
model1_pred_vals_0 = [i[0] for i in model1_pred_vals]
model1_pred_vals_1 = [i[1] for i in model1_pred_vals]

reshaped_model2_pred_vals_0 = [i[0] for i in model2_pred_vals]
reshaped_model2_pred_vals_1 = [i[1] for i in model2_pred_vals]

reshaped_model3_pred_vals_0 = [i[0] for i in model3_pred_vals]
reshaped_model3_pred_vals_1 = [i[1] for i in model3_pred_vals]

reshaped_model4_pred_vals_0 = [i[0] for i in model4_pred_vals]
reshaped_model4_pred_vals_1 = [i[1] for i in model4_pred_vals]

In [ ]:
# Test ANN

import pandas as pd
true_train_classes=validate_first_train.classes
dataset3_train = pd.DataFrame({'first0': model1_pred_vals_0,
                        'first1': model1_pred_vals_1,
                        'inverse0': reshaped_model2_pred_vals_0,
                        'inverse1': reshaped_model2_pred_vals_1,
                        'invverse0': reshaped_model3_pred_vals_0,
                        'invverse1': reshaped_model3_pred_vals_1,
                        'correct':true_train_classes}, 
                       index=np.arange(len(true_train_classes)),columns=['first0',
                                                                   'first1',
                                                                   'inverse0',
                                                                   'inverse1',
                                                                   'invverse0',
                                                                   'invverse1',
                                                                   'correct'])

# Test ANN
import pandas as pd
true_train_classes=validate_first_train.classes
dataset4_train = pd.DataFrame({'first0': model1_pred_vals_0,
                        'first1': model1_pred_vals_1,
                        'inverse0': reshaped_model2_pred_vals_0,
                        'inverse1': reshaped_model2_pred_vals_1,
                        'invverse0': reshaped_model3_pred_vals_0,
                        'invverse1': reshaped_model3_pred_vals_1,
                        'invvverse0': reshaped_model4_pred_vals_0,
                        'invvverse1': reshaped_model4_pred_vals_1,
                        'correct':true_train_classes}, 
                       index=np.arange(len(true_train_classes)),columns=['first0',
                                                                   'first1',
                                                                   'inverse0',
                                                                   'inverse1',
                                                                   'invverse0',
                                                                   'invverse1',
                                                                   'invvverse0',
                                                                   'invvverse1',
                                                                   'correct'])

In [ ]:
X_train_vals = dataset3_train.iloc[:, :6].values
y_train_vals = dataset3_train.iloc[:, 6].values

X_train_vals10 = dataset4_train.iloc[:, :8].values
y_train_vals10 = dataset4_train.iloc[:, 8].values

In [ ]:
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
X_train3, X_test3, y_train3, y_test3 = train_test_split(X_train_vals, y_train_vals, test_size=0.2, random_state=91)
y_train3_cat = np_utils.to_categorical(y_train3,2)
y_test3_cat = np_utils.to_categorical(y_test3,2)

X_train10, X_test10, y_train10, y_test10 = train_test_split(X_train_vals10, y_train_vals10, test_size=0.2, random_state=91)
y_train3_cat10 = np_utils.to_categorical(y_train10,2)
y_test3_cat10 = np_utils.to_categorical(y_test10,2)

In [ ]:
# Creates ann_model2- all training images
from keras.callbacks import ModelCheckpoint

ann_model4 = Sequential()

ann_model4.add(Dense(8, input_dim=8, activation='relu'))
ann_model4.add(Dense(256, activation='relu'))
ann_model4.add(Dropout(0.6))

ann_model4.add(Dense(256, activation='relu'))
ann_model4.add(Dropout(0.6))

# Last layer simple sigmoid function to output 0 or 1 (our label)
ann_model4.add(Dense(2, activation='softmax'))

ann_model4.compile(Adam(lr=0.01),loss='categorical_crossentropy',metrics=['accuracy'])

checkpoint_ann = ModelCheckpoint('ann_three_models4.h5', verbose=1, monitor='loss',save_best_only=True, mode='auto')  

ann_model_results = ann_model4.fit(X_train10,y_train3_cat10,epochs=15,  callbacks=[checkpoint_ann], verbose=1)

In [ ]:
from keras.callbacks import ModelCheckpoint

ann_model = Sequential()

ann_model.add(Dense(6, input_dim=6, activation='relu'))
ann_model.add(Dense(256, activation='relu'))
ann_model.add(Dropout(0.6))

ann_model.add(Dense(256, activation='relu'))
ann_model.add(Dropout(0.6))

# Last layer simple sigmoid function to output 0 or 1 (our label)
ann_model.add(Dense(2, activation='softmax'))

ann_model.compile(Adam(lr=0.01),loss='categorical_crossentropy',metrics=['accuracy'])

checkpoint_ann = ModelCheckpoint('ann_three_models.h5', verbose=1, monitor='loss',save_best_only=True, mode='auto')  

ann_model_results = ann_model.fit(X_train3,y_train3_cat,epochs=15,  callbacks=[checkpoint_ann], verbose=1)

In [ ]:
ann_model4 = load_model('ann_three_models4.h5')

In [ ]:
thirdtest.reset()
model1_pred_test_vals2 = resnet1.predict_generator(thirdtest,steps=thirdtest.samples/thirdtest.batch_size)
thirdtest.reset()
model2_pred_test_vals2 = resnet2.predict_generator(thirdtest,steps=thirdtest.samples/thirdtest.batch_size)
thirdtest.reset()

model3_pred_test_vals2 = resnet3.predict_generator(thirdtest,steps=thirdtest.samples/thirdtest.batch_size)

model1_pred_test_vals_02 = [i[0] for i in model1_pred_test_vals2]
model1_pred_test_vals_12 = [i[1] for i in model1_pred_test_vals2]

reshaped_model2_pred_test_vals_02 = [i[0] for i in model2_pred_test_vals2]
reshaped_model2_pred_test_vals_12 = [i[1] for i in model2_pred_test_vals2]

reshaped_model3_pred_test_vals_02 = [i[0] for i in model3_pred_test_vals2]
reshaped_model3_pred_test_vals_12 = [i[1] for i in model3_pred_test_vals2]

import pandas as pd
true_test_classes2 = thirdtest.classes
dataset32 = pd.DataFrame({'first0': model1_pred_test_vals_02,
                        'first1': model1_pred_test_vals_12,
                        'inverse0': reshaped_model2_pred_test_vals_02,
                        'inverse1': reshaped_model2_pred_test_vals_12,
                        'invverse0': reshaped_model3_pred_test_vals_02,
                        'invverse1': reshaped_model3_pred_test_vals_12,
                        'correct':true_test_classes2}, 
                       index=np.arange(len(true_test_classes2)),columns=['first0',
                                                                   'first1',
                                                                   'inverse0',
                                                                   'inverse1',
                                                                   'invverse0',
                                                                   'invverse1',
                                                                   'correct'])

In [ ]:
X_test_vals2 = dataset3.iloc[:, :8].values
y_test_vals2 = dataset3.iloc[:, 8].values

In [ ]:
# ANN on firsttrain split test imgs
from sklearn.metrics import confusion_matrix,classification_report

predictions_test = ann_model.predict_classes(X_test3)

print(classification_report(y_test3,predictions_test))
print(confusion_matrix(y_test3,predictions_test))

In [ ]:
# ANN on firsttrain split test imgs
from sklearn.metrics import confusion_matrix,classification_report

predictions_test = ann_model4.predict_classes(X_test10)

print(classification_report(y_test10,predictions_test))
print(confusion_matrix(y_test10,predictions_test))

In [ ]:
final_predictions_ann = ann_model4.predict_classes(X_test_vals2)

print(classification_report(y_test_vals2, final_predictions_ann))  
print(confusion_matrix(y_test_vals2,final_predictions_ann))